In [ ]:
import numpy as np

## 创建数组的技巧


1. 使用numpy提供的特殊数组创建函数，比先创建Python对象后再转为数组性能要好。

  如创建一个内含一万个元素的数组：

In [ ]:
%time np.array(range(10000))

CPU times: user 1.49 ms, sys: 4 µs, total: 1.5 ms
Wall time: 1.58 ms


array([   0,    1,    2, ..., 9997, 9998, 9999])

In [ ]:
%time np.arange(10000)

CPU times: user 483 µs, sys: 0 ns, total: 483 µs
Wall time: 401 µs


array([   0,    1,    2, ..., 9997, 9998, 9999])

2. 可以先创建一维数组，在reshape到需要的维度

  如创建一个2行5列的数组，其中的元素为连续整数（0~9）

In [ ]:
np.arange(10).reshape(2, 5)

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

3. 可以使用`numpy.random`模块中的各个函数创建随机数数组

  https://numpy.org/doc/stable/reference/random/index.html

In [ ]:
# 3行2列的随机数数组，元素取值[0.0, 1.0)
np.random.random(size=(3, 2))

array([[0.31160914, 0.52468302],
       [0.6346465 , 0.52686678],
       [0.16584551, 0.9687702 ]])

In [ ]:
# 3行2列的随机数数组，元素取值为[10, 35)的整数
np.random.randint(low=10, high=35, size=(3, 2))

array([[26, 22],
       [29, 20],
       [34, 30]])

In [ ]:
# 也可以从指定元素中抽取(有放回抽样)
np.random.choice(a=['a', 'b', 'c', 'd'], size=(3, 2))

array([['a', 'd'],
       ['c', 'd'],
       ['d', 'a']], dtype='<U1')

In [ ]:
# 或者打乱已有数组（无放回抽样），此时虽不能指定size，如有必要可以切片和reshape
np.random.permutation(['a', 'b', 'c', 'd'])

array(['a', 'b', 'c', 'd'], dtype='<U1')

4. 各种特殊数组的创建方式可以参考：https://numpy.org/doc/stable/reference/routines.array-creation.html

## 数组的基本计算

多维数组之间可以做各种运算，运算是对位运算（矢量化运算），因此参与运算的两个数组shape一般应当相同

In [ ]:
a = np.arange(1, 10).reshape(3, 3)
b = np.arange(10, 100, step=10).reshape(3, 3)

In [ ]:
print(a)
print(b)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[10 20 30]
 [40 50 60]
 [70 80 90]]


In [ ]:
print(a + b)

[[11 22 33]
 [44 55 66]
 [77 88 99]]


In [ ]:
# 需要注意，数组不是矩阵，数组运算（特别是乘法），不是矩阵运算
print(a * b)

[[ 10  40  90]
 [160 250 360]
 [490 640 810]]


数组shape不同时，可能导致运算错误

In [ ]:
np.arange(0, 10).reshape(2, 5) + np.arange(0, 100, step=10).reshape(5, 2)

ValueError: ignored

但是也有特殊情况，比如，数组总能与标量进行计算：

In [ ]:
np.ones((3, 5, 7)) * 10

array([[[10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.]],

       [[10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.]],

       [[10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10.]]])

### 扩展知识——广播

这个特殊情况还可以推广：当两个数组shape不同，但是右对齐shape后能满足下列条件之一时，NumPy会通过广播（broadcast），统一两个数组的shape，再进行计算：

1. 该维度的shape值相同
2. 该维度shape值不同但其中一个为1，该维统一到非1的值
3. 该维度shape值不同但其中一个缺失，该维统一到不缺失的值

如两个数组，shape分别为(3, 5, 7)和(1, 7)，则

* 右对齐shape，即
   ```
   3 x 5 x 7
       1 x 7
   ```
* 从右到左扫描：

  * 最低维，7 == 7，满足条件1；
  * 第二维，5 != 1，但其中一个值是1，满足条件2，统一到5；
  * 最高维，3 != 空, 但其中一个值缺失，统一到3

  因此两个数组是可以计算的。NumPy将(1, 7)的数组通过复制，变为(3, 5, 7)的数组后，再进行计算

In [ ]:
x = np.ones((3, 5, 7))
y = np.arange(7).reshape(1, 7)
print('x=\n', x, sep='')
print('y=\n', y, sep='')
print('x + y =\n', x+y, sep='')

x=
[[[1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]]
y=
[[0 1 2 3 4 5 6]]
x + y =
[[[1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]]

 [[1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]]

 [[1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]
  [1. 2. 3. 4. 5. 6. 7.]]]


## 通用函数

NumPy定义了一系列用于统计计算的函数，称为通用函数，完整列表可见：https://numpy.org/doc/stable/reference/ufuncs.html#available-ufuncs

In [ ]:
arr = np.arange(8 * 5 * 3).reshape(8, 5, 3)
print(arr)

[[[  0   1   2]
  [  3   4   5]
  [  6   7   8]
  [  9  10  11]
  [ 12  13  14]]

 [[ 15  16  17]
  [ 18  19  20]
  [ 21  22  23]
  [ 24  25  26]
  [ 27  28  29]]

 [[ 30  31  32]
  [ 33  34  35]
  [ 36  37  38]
  [ 39  40  41]
  [ 42  43  44]]

 [[ 45  46  47]
  [ 48  49  50]
  [ 51  52  53]
  [ 54  55  56]
  [ 57  58  59]]

 [[ 60  61  62]
  [ 63  64  65]
  [ 66  67  68]
  [ 69  70  71]
  [ 72  73  74]]

 [[ 75  76  77]
  [ 78  79  80]
  [ 81  82  83]
  [ 84  85  86]
  [ 87  88  89]]

 [[ 90  91  92]
  [ 93  94  95]
  [ 96  97  98]
  [ 99 100 101]
  [102 103 104]]

 [[105 106 107]
  [108 109 110]
  [111 112 113]
  [114 115 116]
  [117 118 119]]]


In [ ]:
np.sum(arr)

7140

In [ ]:
# 一部分常用的函数有快捷方式，可以通过数组的成员方法直接调用
arr.sum()

7140

结合矢量化运算和通用函数，可以实现很多功能。

例如，有如下数组，判读其中的元素是否单调递增

In [ ]:
arr1 = np.array([3, 7, 2, 1])
arr2 = np.array([1, 2, 3, 4])

In [ ]:
arr1[1:] > arr1[:-1]

array([ True, False, False])

In [ ]:
np.all(arr1[1:] > arr1[:-1])

False

In [ ]:
np.all(arr2[1:] > arr2[:-1])

True

再来研究如下求平均值的例子，这个过程在实际的数据分析工作中有实际价值吗？

In [ ]:
np.mean(arr)

59.5

In [ ]:
arr.shape

(8, 5, 3)

不妨给数据一点实际含义：

某班级现有5名学生，每个学生参加三门课程：语文、数学和英语。本学期每门课程都进行了8次随堂测验。把测验结果放到一起，就形成了一个`8（次）x 5（人） x 3（门）`的数组。

1. 各个学生各门课程本学期测验平均分是多少？

  对于这个问题，应该得到一个`5（人） x 3（门）`的结果，即通过统计汇总，消除掉8（次）的这个维度（第0维）。简称对第0维求平均值。

In [ ]:
np.mean(arr, axis=0)

array([[52.5, 53.5, 54.5],
       [55.5, 56.5, 57.5],
       [58.5, 59.5, 60.5],
       [61.5, 62.5, 63.5],
       [64.5, 65.5, 66.5]])

2. 这个班级每次测验每门课程的班级平均分是多少？

  这个问题要消除的是学生维度，即对第1维求平均值。

3. 每名学生每次测验的平均分是多少？

4. 每名学生本学期的总平均分是多少？

  这个问题需要略作分析，最终答案应该只和学生人数有关，即shape=5（人），则要同时消去0维和2维。

### 答案依次如下：

In [ ]:
np.mean(arr, axis=1)

array([[  6.,   7.,   8.],
       [ 21.,  22.,  23.],
       [ 36.,  37.,  38.],
       [ 51.,  52.,  53.],
       [ 66.,  67.,  68.],
       [ 81.,  82.,  83.],
       [ 96.,  97.,  98.],
       [111., 112., 113.]])

In [ ]:
np.mean(arr, axis=2)

array([[  1.,   4.,   7.,  10.,  13.],
       [ 16.,  19.,  22.,  25.,  28.],
       [ 31.,  34.,  37.,  40.,  43.],
       [ 46.,  49.,  52.,  55.,  58.],
       [ 61.,  64.,  67.,  70.,  73.],
       [ 76.,  79.,  82.,  85.,  88.],
       [ 91.,  94.,  97., 100., 103.],
       [106., 109., 112., 115., 118.]])

In [ ]:
np.mean(arr, axis=(0, 2))

array([53.5, 56.5, 59.5, 62.5, 65.5])

也可以分两步进行计算，但要注意顺序

In [ ]:
arr.mean(axis=2).mean(axis=0)

array([53.5, 56.5, 59.5, 62.5, 65.5])

In [ ]:
arr.mean(axis=0).mean(axis=1)

array([53.5, 56.5, 59.5, 62.5, 65.5])

## 基于条件的筛选——布尔值索引

In [ ]:
names = np.array(['bob', 'joe', 'will', 'bob', 'will', 'joe', 'joe'])
data = np.array([71, 89, 91, 60, 78, 93, 82])

如何得到属于`bob`的所有数据？

In [ ]:
# NumPy除了下标索引外，还支持布尔值索引。索引的shape必须和数组的shape相等，选出索引为True对应位置上的元素。如要选择第0、第3个元素：
data[[True, False, False, True, False, False, False]]

array([71, 60])

In [ ]:
# 结合矢量化布尔运算和布尔值索引，就能实现条件筛选
names == 'bob'

array([ True, False, False,  True, False, False, False])

In [ ]:
data[names == 'bob']

array([71, 60])